In [35]:
from lazypredict.Supervised import LazyClassifier
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.inspection import DecisionBoundaryDisplay

from scipy import stats
from pca import pca
from IPython.display import display
import dataframe_image as dfi

from src.visualization import feature_importances_plot

import warnings
    
# warnings -> to silence warnings

warnings.filterwarnings("ignore")
np.set_printoptions(precision=5, suppress=True)


RANDOM_STATE = 42
N_JOBS = -1
class_names = ["Canis", "Dysg. Equisimilis", "Dysg. Dysgalactiae"]

map_target = {
    "Streptococcus canis": 0,
    "Streptococcus dysgalactiae subsp. equisimilis": 1,
    "Streptococcus dysgalactiae subsp. dysgalactiae": 2
}

map_target_inv = {
    0: "Strept. canis",
    1: "Strept. dysg. equisimilis",
    2: "Strept. dysg. dysgalactiae"
}

map_target_antibiotici = {
    "S" : 1,
    "NS" : 0
}
start = 9
n_antibiotici = 9
n_geni = 27
n_virulenza = 18
n_picchi = ['46','306']
#n_picchi = ['46']
N_BEST = 3

In [36]:
def makeDictBest(models, modelli, n_best = 5):
    for i in range(n_best):
        best = models.index[i]
        score = modelli.get(best)
        if score != None:
            modelli[best] = score + (n_best-i)
        else:
            modelli[best] = (n_best-i)
    return modelli

In [37]:
for n in n_picchi:
    print('DATAFRAME CON '+n+' PICCHI')
    df = pd.read_csv("../data/Dati_Matemaldomics_"+n+"picchi.csv",
                    delimiter=';', index_col='ID Strain')
    n = int(n)
    modelli = {}
    animal  = df[['Animal species of origin']]
    lancefield = df[['LANCEFIELD GROUP']]
    haemolysis = df[['Haemolysis']]
    subspecies = df[['Putative Subspecies']]

    st = df[[df.columns[4]]]
    maldi = df[df.columns[start:start+n]]
    antibiotici = df[df.columns[start+n:start+n+n_antibiotici]]
    geni_antibiotici = df[df.columns[start+n+n_antibiotici:start+n+n_antibiotici+n_geni]]
    virulenza = df[df.columns[start+n+n_antibiotici+n_geni:start+n+n_antibiotici+n_geni+n_virulenza]]
    
    maldi.fillna(0, inplace=True)
    maldi = maldi.replace(',', '.', regex=True)
    columns = maldi.columns
    for column in columns:
        maldi[column] = maldi[column].astype(float)
    display(maldi)
    
    targets = {'antibiotici' : antibiotici,
                'geni_antibiotici' : geni_antibiotici,
                'virulenza' : virulenza}
    
    feats_agg = {'lancefield' : lancefield,
                'haemolysis' : haemolysis,
                'subspecies' : subspecies,        
                'animal' : animal}
    
    for str_target,target in targets.items():
        columns = target.columns
        for column in columns:
            if str_target == 'antibiotici':
                target[column] = df[column].map(map_target_antibiotici)
            rapporto = (target[column] == 0).sum() / target.shape[0]
            #if (antibiotici[column] == 0).all() or (antibiotici[column] == 1).all():
            print(column+" : "+str(rapporto))
            if rapporto < 0.15 or rapporto > 0.85:
                target.drop([column], axis=1, inplace=True)
        
        display(target)
        
    metrics_df = pd.DataFrame(columns=['Target','Model','Accuracy','Precision','Recall','F1-Score','CV'])
    metrics_df_agg = pd.DataFrame(columns=['Target','Model','Accuracy','Precision','Recall','F1-Score','CV'])
    skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    targets['subspecies'] = subspecies
    targets['st'] = st
    X = maldi
    for str_target, target in targets.items():
        columns = target.columns
        for column in columns:    
            y = target[column]
            n_classes = np.unique(y)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
            clf = LazyClassifier(predictions=True)
            models, predictions = clf.fit(X_train, X_test, y_train, y_test)
            print("Colonna:"+column)
            display(models)
            print("\n")
            modelli = makeDictBest(models, modelli)
            
            models.to_csv('Risultati/LazyPredictor/model_'+str(n)+column+'.csv')
    del targets['subspecies']
    for str_feat, feat_agg in feats_agg.items():
        display(feat_agg)
        X = pd.concat([X, feat_agg], axis=1)
        for str_target, target in targets.items():
            columns = target.columns
            for column in columns:    
                y = target[column]
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
                clf = LazyClassifier(predictions=True)
                models, predictions = clf.fit(X_train, X_test, y_train, y_test)
                print("Colonna: "+column+" con feat agg: "+str_feat)
                display(models)
                print("\n")
                modelli = makeDictBest(models, modelli)
                
                models.to_csv('Risultati/LazyPredictor/model_'+str(n)+column+'_morefeat'+str_feat+'.csv')
    modelli = sorted(modelli.items(), key=lambda x:x[1])
    print(modelli)

DATAFRAME CON 46 PICCHI


,"2223,140967","2241,073989","2262,75751","2679,802856","2978,296408","3159,441237","3354,28405","3364,608472","3397,909861","3418,174965",...,"9030,351844","9073,208159","9487,183195","10103,20284","10400,80576","10491,16654","10930,54833","13276,73249","14943,03835","15048,89449"
ID Strain,,,,,,,,,,,,,,,,,,,,,
V13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V142,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V151,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V160,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V161,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V800,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
V90,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Eritromicina : 0.461038961038961
Ceftiofur : 0.0
Tetraciclina : 0.5194805194805194
Gentamicina : 0.6233766233766234
Penicillina : 0.0
Ampicillina : 0.0
Sulfametossazolo_trimethoprim : 0.01948051948051948
Clindamicina : 0.2662337662337662
Enrofloxacin : 0.6688311688311688


,Eritromicina,Tetraciclina,Gentamicina,Clindamicina,Enrofloxacin
ID Strain,,,,,
V13,0,0,0,1,0
V142,0,1,1,1,0
V151,1,1,0,1,0
V160,1,0,0,1,0
V161,1,1,0,1,0
...,...,...,...,...,...
V800,1,0,1,1,0
V82,1,1,0,1,1
V90,1,0,0,1,0


aad(6) : 0.935064935064935
ANT(6)-Ia : 0.8246753246753247
APH(2'')-IIIa : 1.0
APH(3')-IIIa : 0.9025974025974026
catS : 0.9675324675324676
dfrF : 0.9805194805194806
E. faecalis chloramphenicol acetyltransferase : 0.9935064935064936
Erm(47) : 0.987012987012987
ErmB : 0.8181818181818182
fexA : 0.9935064935064936
L._reuteri cat-TC : 1.0
lmrP : 0.006493506493506494
lnuC : 0.987012987012987
lnuD : 0.9935064935064936
lsaC : 0.961038961038961
lsaE : 0.7857142857142857
mefE : 0.8506493506493507
optrA : 0.9935064935064936
poxtA : 0.9935064935064936
SAT-4 : 0.922077922077922
tet(40) : 0.987012987012987
tet(L) : 0.9935064935064936
tetM : 0.8181818181818182
tetO : 0.7402597402597403
tetS : 0.9805194805194806
tetT : 0.974025974025974
vatE : 0.9935064935064936


...

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.84,0.82,0.82,0.84,0.41
RidgeClassifier,0.84,0.82,0.82,0.84,0.09
BaggingClassifier,0.87,0.81,0.81,0.87,0.21
ExtraTreeClassifier,0.81,0.80,0.80,0.81,0.08
RidgeClassifierCV,0.84,0.79,0.79,0.84,0.11
NearestCentroid,0.74,0.79,0.79,0.75,0.09
SGDClassifier,0.81,0.77,0.77,0.81,0.09
BernoulliNB,0.74,0.75,0.75,0.75,0.09
Perceptron,0.77,0.74,0.74,0.78,0.10


100%|██████████| 29/29 [00:14<00:00,  1.95it/s]

Colonna: ST con feat agg: animal


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
RandomForestClassifier,0.52,0.44,None,0.46,0.79
ExtraTreesClassifier,0.55,0.42,None,0.50,0.56
LGBMClassifier,0.48,0.38,None,0.46,5.96
LinearDiscriminantAnalysis,0.39,0.30,None,0.42,0.15
BernoulliNB,0.35,0.27,None,0.31,0.09
DecisionTreeClassifier,0.32,0.24,None,0.32,0.20
RidgeClassifierCV,0.29,0.23,None,0.26,0.12
LinearSVC,0.29,0.23,None,0.26,0.51
BaggingClassifier,0.39,0.23,None,0.37,0.81




[('CalibratedClassifierCV', 7), ('KNeighborsClassifier', 12), ('GaussianNB', 14), ('BaggingClassifier', 18), ('AdaBoostClassifier', 19), ('SGDClassifier', 22), ('LinearDiscriminantAnalysis', 29), ('QuadraticDiscriminantAnalysis', 29), ('ExtraTreeClassifier', 31), ('LGBMClassifier', 33), ('PassiveAggressiveClassifier', 37), ('SVC', 38), ('Perceptron', 38), ('DecisionTreeClassifier', 41), ('ExtraTreesClassifier', 42), ('RandomForestClassifier', 44), ('LinearSVC', 46), ('XGBClassifier', 47), ('RidgeClassifierCV', 57), ('LogisticRegression', 61), ('NearestCentroid', 65), ('RidgeClassifier', 66), ('BernoulliNB', 119)]
